In [5]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)


def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()


def comment_on_own_page(user_id):
    # select user.id, comment.content from comment join user on user.id = comment.author_id where comment.uuid_id = user.id;
    return len(query(f"""select comment_vote.user_id
    from comment_vote
    inner join comment
    on comment.id = comment_vote.comment_id
    where comment_vote.user_id = comment.author_id and comment_vote.user_id = {user_id}""")) > 0
    
    
df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df["comment_on_own_page"] = df["id"].map(comment_on_own_page)  
df_potential_bot = df[
    ((df["edits"] + df["reviews"] + df["taxonomy"] == 0) & (df["comments"] > 0) & df["comment_on_own_page"])
].copy()
df_potential_bot.sort_values(["id"], inplace=True)
df_potential_bot.to_csv("/tmp/2022-21-03users_with_comments_on_own_page.csv")
df_potential_bot


,id,username,description,date,edits,reviews,comments,taxonomy,comment_on_own_page
1856,39586,Beachy,NULL,2015-04-24 09:01:24,0.0,0.0,1.0,0.0,True
2250,47567,TheGenius,NULL,2015-10-04 09:22:25,0.0,0.0,1.0,0.0,True
2573,51931,rmg,,2016-02-24 14:25:17,0.0,0.0,2.0,0.0,True
2661,54733,WolfOliver,NULL,2016-03-18 10:33:11,0.0,0.0,3.0,0.0,True
2990,61630,amitsulu,NULL,2016-08-30 06:49:01,0.0,0.0,3.0,0.0,True
3082,64299,DoDoR,NULL,2016-10-05 13:33:36,0.0,0.0,1.0,0.0,True
3691,67431,HornyUnicorn,NULL,2016-12-31 15:53:59,0.0,0.0,11.0,0.0,True
3767,68040,checker001,NULL,2017-01-20 07:56:31,0.0,0.0,1.0,0.0,True
3976,69781,malte5101,NULL,2017-02-28 07:33:52,0.0,0.0,13.0,0.0,True
4275,73761,Sarittaa04,NULL,2017-04-04 23:30:20,0.0,0.0,12.0,0.0,True
